<a href="https://colab.research.google.com/github/Shopping-Yuan/ML_Project/blob/Shopping_vscode_branch/Decision_Tree/CART_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import titanic dataset from kaggle



In [5]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240529%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240529T115735Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D480e2d38ebcca4bcf2596f239480dacea5e218fafa2d6c8123570acf03710eafde3e6e5ca5ce499f8bff954eb435254b98dbd4427e2e4231af953cf571ecd6b3b590e84574ce70c15c787d2d7eddb7d94b12d745c61b81e8e550af0cb0ce52a136a9e86b2060767168b0fa86168ba5f51233b7c9c21834ae777e6857fdd8e6ba921986e551ec966ef3494ce06f9a9409c7731a3076ba115eeb5af5e2f146204ebc777862913db43da9d351495db65fc67592b37148543b7b3353ccf07a416651be60142a70d91be8165cce7bf435ff1d943d23c52767b203dcf54afd299e65492221624f35f0a1657f12ef7d8c2319476299ca8520ae777aaa3503fe4cf9817e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 34877 bytes downloaded
Downloaded and uncompressed: titanic
Data source import complete.


In [6]:
import pandas as pd
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [45]:
from sklearn.model_selection import train_test_split
train_path = "/kaggle/input/titanic/train.csv"
test_path = "/kaggle/input/titanic/test.csv"

def get_datasets(train_path,test_path,random_forest = True):
  train_data = pd.read_csv(train_path)
  test_data = pd.read_csv(test_path)

  train_data = train_data.dropna().drop_duplicates()

  test_data = test_data.dropna().drop_duplicates()

  drop_features = ["PassengerId","Name","Cabin","Ticket"]

  if not random_forest :

    train_data = train_data.drop(axis=0,columns=drop_features,errors='raise')
    test_data = test_data.drop(axis=0,columns=drop_features,errors='raise')

    train_data,val_data= \
    train_test_split(train_data, test_size=0.2, random_state=100, stratify = train_data["Survived"])
    return train_data, val_data, test_data
  else :
    list_of_train_data, list_of_val_data = bootstrapping_oob(train_data,drop_features)
    original_train_data = train_data.drop(axis=0,columns=drop_features,errors='raise')
    return list_of_train_data, list_of_val_data, test_data, original_train_data

In [55]:
from random import choices
def bootstrapping_oob(train_data,drop_feature,tree_number = 100, sample_number = None):
  if not sample_number:
    sample_number = train_data.shape[0]
  list_of_train_data = []
  list_of_val_data = []
  for n in range(tree_number):
    # bootstrapping
    index = list(range(train_data.shape[0]))
    train_set_index = choices(index, k = sample_number)
    nth_train_set = pd.DataFrame([train_data.iloc[i] for i in train_set_index])

    # oob_validation : train_data - nth_train_set
    nth_oob_set = pd.concat([train_data,nth_train_set]).drop_duplicates(keep=False)

    nth_train_set = nth_train_set.drop(axis=0,columns=drop_feature,errors='raise')
    nth_oob_set = nth_oob_set.drop(axis=0,columns=drop_feature,errors='raise')
    list_of_train_data.append(nth_train_set)
    list_of_val_data.append(nth_oob_set)

  return list_of_train_data,list_of_val_data

In [10]:
# train_data = pd.read_csv(train_path).dropna().drop_duplicates()
# drop_features = ["PassengerId","Name","Cabin","Ticket"]
# bootstrapping_oob(train_data,drop_features,tree_number = 1, sample_number = None)
# print(train_data[train_data.duplicated()])

In [11]:
class DecisionStump():
    def __init__(self,partition_num = 10):

      self.partition_num = partition_num

    def fit(self, data,feature_name,target_name):

      min_value = data[feature_name].min()
      max_value = data[feature_name].max()
      step = (max_value - min_value)/self.partition_num
      partitions = [min_value + step * i for i in range(1,self.partition_num-1)] + [max_value]

      df = data.sort_values(by = feature_name)

      best_threshold = 0
      best_error = df.shape[0]
      best_df1 = df
      best_df2 = df
      best_direction = None
      for threshold in partitions:

        df1 = df[df[feature_name]>=threshold]
        df1_class1 = df1[df1[target_name] == 0].shape[0]
        df1_class2 = df1[df1[target_name] == 1].shape[0]

        df2 = df[df[feature_name]<threshold]
        df2_class1 = df2[df2[target_name] == 0].shape[0]
        df2_class2 = df2[df2[target_name] == 1].shape[0]

        if (df1_class1 + df2_class2) >= (df1_class2 + df2_class1):
          error = (df1_class2 + df2_class1)
          direction = "p"
        else :
          error = (df1_class1 + df2_class2)
          direction = "n"
        if error < best_error :
          best_threshold = threshold
          best_df1 = df1
          best_df2 = df2
          best_direction = direction

      return best_threshold,best_df1,best_df2,best_direction

In [12]:
import math
def gini_impurity_test(df1, df2 , target_name):
    names_1 = df1[target_name].unique()[0]
    names_2 = df2[target_name].unique()[0]

    n1 = df1.shape[0]
    n2 = df2.shape[0]
    p_1 = df1[df1[target_name] == names_1].shape[0] / n1
    p_2 = df2[df2[target_name] == names_2].shape[0] / n2
    g_1 = 2*p_1*(1-p_1)
    g_2 = 2*p_2*(1-p_2)

    gini = (n1 * g_1 + n2 * g_2)/(n1 + n2)
    return gini
def max_information_gain(df1, df2 , target_name, relative = True):
    # display(df1)
    # display(df2)
    names_1 = df1[target_name].unique()[0]
    names_2 = df2[target_name].unique()[0]
    n1 = df1.shape[0]
    n2 = df2.shape[0]
    p_1 = df1[df1[target_name] == names_1].shape[0] / n1
    p_2 = df2[df2[target_name] == names_2].shape[0] / n2

    E_A1 = -1 * (p_1*math.log2(p_1)+(1-p_1)*math.log2(p_1))
    E_A2 = -1 * (p_2*math.log2(p_2)+(1-p_2)*math.log2(p_2))
    H_DA = (n1 * E_A1 + n2 * E_A2)/(n1 + n2)
    total = pd.concat([df1, df2])
    p_total = total[total[target_name] == names_1].shape[0]/(n1 + n2)
    H_D = -1*(p_total*math.log2(p_total)+(1-p_total)*math.log2(p_total))

    if relative == True :
      if H_DA == 0:
        return -100

      return -1*(H_D - H_DA)/H_DA
    else :
      return -1*(H_D - H_DA)




# train_df = train_data.join(train_target)
# df1 = train_df[train_df["Sex"] == "male"]
# df2 = train_df[train_df["Sex"] == "female"]
# target_name = "Survived"

# display(train_df.head())
# print(gini_impurity_test(df1, df2 , target_name))

In [13]:
class Subtree():
  def __init__(self,data,target_name,feauture_pair):
    self.data = data
    self.target_name = target_name
    self.feauture_pair = feauture_pair
    self.parent = None
    self.right = None
    self.left = None
    self.feature = None
    self.c = None
    self.threshold = None
    self.direction = None
    self.outcome = None

  def _feature_selection(self,feature_select_algorithm):
    best_error = 0.5
    b_l = None
    b_r = None
    b_f = None
    b_t = None
    b_d = None
    for name, f_type in self.feauture_pair:
      threshold = None
      direction = None
      if f_type == "Quantitative":
        classifier = DecisionStump(partition_num = 10)
        threshold,left_data,right_data,direction = \
        classifier.fit(self.data,name,self.target_name)
        if left_data.empty or right_data.empty:
            continue
        error = feature_select_algorithm(left_data,right_data,self.target_name)
        if best_error >= error:
          best_error = error
          b_l, b_r, b_f,b_t,b_d = left_data,right_data,name, threshold, direction

      elif f_type == "Qualitative":
        for c in self.data[name].unique():
          left_data = self.data[self.data[name] == c]
          right_data = self.data[self.data[name] != c]
          if left_data.empty or right_data.empty:
            continue
          error = feature_select_algorithm(left_data,right_data,self.target_name)
          if best_error >= error:
            best_error = error
            self.c = c
            b_l, b_r, b_f,b_t,b_d = left_data,right_data,name, threshold, direction
    # print(b_l,b_r)
    return b_l,b_r,b_f,b_t,b_d
  def _get_subtree(self,feature_select_algorithm):
    left_data,right_data,self.feature,self.threshold,self.direction = \
    self._feature_selection(feature_select_algorithm)

    self.left,self.right = \
    Subtree(left_data,self.target_name,self.feauture_pair),\
    Subtree(right_data,self.target_name,self.feauture_pair)

    # print(f"left {left_data}")
    # print(f"right {right_data}")

    self.left.parent = self
    self.right.parent = self

  def get_all_subtrees(self,feature_select_algorithm):
    if self.data.shape[0] ==1 or \
      len(self.data[self.target_name].unique()) == 1 :
      self.outcome = self.data[self.target_name].unique()[0]
    else :
      self._get_subtree(feature_select_algorithm)
      self.left.get_all_subtrees(feature_select_algorithm)
      self.right.get_all_subtrees(feature_select_algorithm)
  def _search_leaf(self,new_data,name):
    if not self.feature:
      return self.outcome
    elif self.c:
      if new_data[self.feature] == self.c:
        return self.left._search_leaf(new_data,name)
      else :
        return self.right._search_leaf(new_data,name)
    else:
      if (self.direction == "p" and new_data[self.feature]>= self.threshold) or \
        (self.direction == "n" and new_data[self.feature]< self.threshold) :
        return self.right._search_leaf(new_data,name)
      else:
        return self.left._search_leaf(new_data,name)
  def search_all_leaf(self,new_data,name):
    new_data[name] = new_data.apply(self._search_leaf,name = name, axis = 1)

In [50]:
class CART_Classification():
  def __init__(self,train_data, val_data, test_data, target_name, feauture_pair):
    self.train_data = train_data
    self.val_data = val_data
    self.test_data = test_data
    self.target_name = target_name
    self.feauture_pair = feauture_pair
  def training(self):
    root = Subtree(self.train_data,self.target_name,self.feauture_pair)
    # root.get_all_subtrees(max_information_gain)
    root.get_all_subtrees(gini_impurity_test)
    return root
  @staticmethod
  def inference(data,name,CART_tree):
    CART_tree.search_all_leaf(data,name)
  @staticmethod
  def get_accuracy(data,name1,name2):
    return len(data[data[name1] == data[name2]])/len(data)



random forest = bagging(bootstrapping ,out of bags set validation) + CART

In [56]:
from tqdm import tqdm
import numpy as np
def main(random_forest = True):

  myseed = 1
  np.random.seed(myseed)

  target_name = "Survived"
  feauture_type = ["Qualitative","Qualitative","Quantitative","Qualitative","Qualitative","Quantitative","Qualitative"]

  if not random_forest:
    # get data set
    train_data, val_data, test_data = get_datasets(train_path,test_path,random_forest)
    feauture_pair = list(zip(train_data.columns.drop("Survived"),feauture_type))
    # training
    CART = CART_Classification(train_data, val_data, test_data, "Survived", feauture_pair)
    CART_train_tree = CART.training()
    # validation
    CART_Classification.inference(val_data,"predict_Survived",CART_train_tree)
    acc = CART_Classification.get_accuracy(val_data,"Survived","predict_Survived")
    print(f"train accuracy = {acc}")
    # using whole dataset to train model again
    total_train = pd.concat([train_data, val_data])
    CART_total = CART_Classification(train_data, val_data, test_data, "Survived", feauture_pair)
    CART_total_train_tree = CART_total.training()
    # inference
    CART_Classification.inference(test_data,"predict_Survived",CART_total_train_tree)
    return [[x] for x in test_data["predict_Survived"].to_list()]
  else :
    # get data set
    list_of_train_data, list_of_val_data, test_data, original_train_data= get_datasets(train_path,test_path,random_forest)
    feauture_pair = list(zip(original_train_data.columns.drop("Survived"),feauture_type))
    list_of_train_trees = []
    list_of_prediction = []
    for t,v in tqdm(list(zip(list_of_train_data,list_of_val_data)),desc = "train and val nth tree"):
      # training of each tree
      CART = CART_Classification(t, v, test_data, "Survived", feauture_pair)
      CART_train_tree = CART.training()
      list_of_train_trees.append(CART_train_tree)
      # validation each tree using their OOB datas
      CART.inference(v,"predict_Survived",CART_train_tree)
      # print(v["predict_Survived"].head(15))
      list_of_prediction.append(v["predict_Survived"])
    # get prediction
    predictions = pd.DataFrame(list_of_prediction)
    # display(predictions.head())
    most_likely_prediction = predictions.mode(axis=0, numeric_only=False, dropna=True)
    most_likely_prediction = most_likely_prediction.iloc[0].dropna()
    index = original_train_data.index.intersection(most_likely_prediction.index)
    label = original_train_data["Survived"].loc[index]
    # print(most_likely_prediction)
    # print(label)
    acc = most_likely_prediction.eq(label).sum()/label.shape[0]
    print(f"train accuracy = {acc}")
    # inference
    list_of_inference = []
    for tree in tqdm(list_of_train_trees,desc = "tree number"):
      CART_Classification.inference(test_data,"predict_Survived",tree)
      list_of_inference.append(test_data["predict_Survived"])
    predictions = pd.DataFrame(list_of_prediction)
    most_likely_prediction = predictions.mode(axis=0, numeric_only=False, dropna=True)
    most_likely_prediction = most_likely_prediction.iloc[0]
    return [[int(x)] for x in most_likely_prediction.to_list()]
main(random_forest = True)

train and val nth tree: 100%|██████████| 100/100 [04:59<00:00,  2.99s/it]


train accuracy = 0.7103825136612022


tree number: 100%|██████████| 100/100 [00:00<00:00, 280.37it/s]


[[1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1]